In [1]:

print('hi')

hi


# Модель выбытия жилищного фонда

Расчитывает выбытия жилой площади на периоде 2015 - 2035 гг.

In [5]:
from os import path, getcwd

import pandas as pd
import numpy as np
import sqlalchemy as sa
import re
import matplotlib.pyplot as plt

strHousesDBPath=path.join('..','DB', 'Houses.sqlite3') # путь к рабочей базе данных SQLite Houses

# константы базы данных SQLite3
strHouse_table='houses' # название таблицы в базе данных SQLite
strHouse_data_pass='houses_columns' # таблица с названиями колонок


## Данные из БД 
<div class="alert alert-block alert-info">
<p>
    <ul>
        <li>забираем данные из БД, в виде DataFrame </li>
        <li>вводятся ограничения на год постройки</li>
        <li>выбираются только данные по МКД</li>
    </ul> 
</p>
</div>

Создаем подключение к БД

In [6]:
strSELECT_House='select * from {house_table}'.format(house_table=strHouse_table)

conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strHousesDBPath)) # connection к рабочей базе данных

pdfHouse=pd.read_sql(strSELECT_House, con=conWork)

Обработка панели:
сортируем данные по году постройки, выбираем с 1801 по 2019

In [7]:
wdf = pdfHouse.copy()
wdf = wdf.sort_values(by=['built_year']) # сортировка по году постройки
wdf = wdf[(wdf['built_year'] >= 1801) & (wdf['built_year'] <= 2019)] # выбор данных с 1801 по 2019
wdf.reset_index(drop=True,inplace=True)
wdf

,houseguid,built_year,house_type,is_alarm,floor_count_max,elevators_count,area_total,area_residential,foundation_type,floor_type,wall_material,heating_type,hot_water_type,cold_water_type,living_quarters_count
0,8df065e7-1bba-4a5e-b1ca-0e84b3b1359c,1801.0,None,Нет,2.0,NaN,169.0,164.9,None,None,None,None,None,None,5.0
1,0dc1ad4f-e383-40f2-9067-6c1122bc4643,1801.0,Многоквартирный дом,Нет,3.0,0.0,616.7,236.6,None,Деревянные,Кирпич,Центральное,Отсутствует,Центральное,NaN
2,ded84f7a-7994-44c9-9064-70b88b91f076,1801.0,Многоквартирный дом,Нет,4.0,0.0,550.3,231.3,Ленточный,Деревянные,Кирпич,Центральное,Закрытая с приготовлением горячей воды на ЦТП,Центральное,15.0
3,f8078873-01a8-420f-a07a-f2179620afcc,1802.0,Многоквартирный дом,Нет,5.0,0.0,8946.0,5265.0,Ленточный,Смешанные,Кирпич,Центральное,Открытая с отбором сетевой воды на горячее вод...,Центральное,0.0
4,e4c23e92-615d-4e1c-93b9-4d10e76c3ca3,1802.0,Многоквартирный дом,Нет,2.0,0.0,542.1,435.9,Ленточный,Деревянные,Кирпич,Центральное,Открытая с отбором сетевой воды на горячее вод...,Центральное,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679983,2afa1b79-645b-416f-88ba-7487c70b8273,2019.0,Многоквартирный дом,Нет,20.0,4.0,12479.5,8239.3,Свайный,Железобетонные,Железобетонная панель,Индивидуальный тепловой пункт (ИТП),Закрытая с приготовлением горячей воды на ИТП,Центральное,219.0
679984,7ca3ed9d-ccf3-440e-bbda-7a680a20433e,2019.0,None,Нет,17.0,2.0,9579.7,7241.2,None,None,None,None,None,None,153.0
679985,e9378f1a-1659-4236-86ff-0c20dc646556,2019.0,None,Нет,14.0,4.0,14226.0,8720.9,None,None,None,None,None,None,139.0
679986,d8287c8c-61b0-40c3-8997-642a72f67c87,2019.0,Многоквартирный дом,Нет,32.0,14.0,56107.4,43597.9,Сплошной,Железобетонные,Монолитные,Индивидуальный тепловой пункт (ИТП),Закрытая с приготовлением горячей воды на ЦТП,Центральное,894.0


In [8]:
wdf['house_type'].unique() # выводит типы домов, которые присутствуют в данных 

array([None, 'Многоквартирный дом', 'Специализированный жилищный фонд',
       'Жилой дом блокированной застройки',
       'Жилой дом (индивидуально-определенное здание)'], dtype=object)

In [9]:
# Оставляем только МКД и Жилые дома блокированной застройки (В данном варианте модели используются только МКД. 
# В некоторых случаях жилые дома блокированной застройки также могут классифицироваться как МКД, 
# поэтому данные по ним оставлены, хотя и не используются.)
wdf=wdf[(wdf['house_type']=='Многоквартирный дом')|
        (wdf['house_type']=='Жилой дом блокированной застройки')]

## Условия

<div class="alert alert-block alert-info">
<p>Создаются условия выбытия:
    <ul>
        <li>9 вариантов для домов различных по материалу несущих стен и, соответственно, расчитанных на различный срок эксплуатации. Причем панельные и кирпичные дома разбиваются на низкоэтажные (до 5 этажей) и высокоэтажные (6 и более) </li>
        <li>1 вариант для любого вида домов, признаных аварийным </li>
    </ul> 

Накладывая каждое условие по почереди на совокупность данных - будет рассчитываться площадь ЖФ, которая подлежит выбытию в каждый год. Например, срок жизни деревянного (многоквартирного) дома составляет 50 лет. Соответственно, в 2020 году из эксплуатации должны выйти дома такого типа, построенные в 1970 г. В 2021 г. - построенные в 1971 г.
    
    
Критерии для условий выбираем исходя из следующих документов (ссылка)
</p>
</div>

Каждое условие записывается в отдельную переменную "cond", содержащую информацию по: доле учитываемого фонда, возрасту, этажности (от и до), аварийности, типу несущих стен и типу дома.

In [10]:
cond1=pd.Series({'Доля учитываемого фонда': 1, 
                 'Возраст': 50,
                 'Аварийный': 'Нет', 
                 'Несущие стены': 'Деревянные', 
                 'Тип дома': 'Многоквартирный дом'}, name='Условие 1')
cond2=pd.Series({'Доля учитываемого фонда': 0.7,
                 'Возраст': 60, 
                 'Этажность (от)': 1,
                 'Этажность (до)': 5,
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Панельные', 
                 'Тип дома': 'Многоквартирный дом'}, name='Условие 2')
cond3=pd.Series({'Доля учитываемого фонда': 0.5,
                 'Возраст': 80, 
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Смешанные', 
                 'Тип дома': 'Многоквартирный дом'}, name='Условие 3')
cond4=pd.Series({'Доля учитываемого фонда': 0.5,
                 'Возраст': 60,
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Иные', 
                 'Тип дома': 'Многоквартирный дом'}, name='Условие 4')
cond5=pd.Series({'Доля учитываемого фонда': 0.3,
                 'Возраст': 100,
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Блочные', 
                 'Тип дома': 'Многоквартирный дом'}, name='Условие 5')
cond6=pd.Series({'Доля учитываемого фонда': 0.3,
                 'Возраст': 100, 
                 'Этажность (от)': 1,
                 'Этажность (до)': 5,
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Кирпич', 
                 'Тип дома': 'Многоквартирный дом'}, name='Условие 6')
cond7=pd.Series({'Доля учитываемого фонда': 0.1,
                 'Возраст': 100, 
                 'Этажность (от)': 6,
                 'Этажность (до)': 50,
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Кирпич', 
                 'Тип дома': 'Многоквартирный дом'}, name='Условие 7')
cond8=pd.Series({'Доля учитываемого фонда': 0.5,
                 'Возраст': 100, 
                 'Этажность (от)': 6,
                 'Этажность (до)': 50,
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Панельные', 
                 'Тип дома': 'Многоквартирный дом'}, name='Условие 8')
cond9=pd.Series({'Доля учитываемого фонда': 0.1,
                 'Возраст': 150,
                 'Аварийный': 'Нет',
                 'Несущие стены': 'Монолитные', 
                 'Тип дома': 'Многоквартирный дом'}, name='Условие 9')
cond10=pd.Series({'Аварийный': 'Да',
                 'Тип дома': 'Многоквартирный дом'}, name='Условие 10')

Cond_df=pd.DataFrame([cond1,cond2,cond3,cond4,cond5,cond6,cond7,cond8,cond9,cond10])

In [13]:
Cond_df

,Доля учитываемого фонда,Возраст,Аварийный,Несущие стены,Тип дома,Этажность (от),Этажность (до)
Условие 1,1.0,50.0,Нет,Деревянные,Многоквартирный дом,NaN,NaN
Условие 2,0.7,60.0,Нет,Панельные,Многоквартирный дом,1.0,5.0
Условие 3,0.5,80.0,Нет,Смешанные,Многоквартирный дом,NaN,NaN
Условие 4,0.5,60.0,Нет,Иные,Многоквартирный дом,NaN,NaN
Условие 5,0.3,100.0,Нет,Блочные,Многоквартирный дом,NaN,NaN
Условие 6,0.3,100.0,Нет,Кирпич,Многоквартирный дом,1.0,5.0
Условие 7,0.1,100.0,Нет,Кирпич,Многоквартирный дом,6.0,50.0
Условие 8,0.5,100.0,Нет,Панельные,Многоквартирный дом,6.0,50.0
Условие 9,0.1,150.0,Нет,Монолитные,Многоквартирный дом,NaN,NaN
Условие 10,NaN,NaN,Да,NaN,Многоквартирный дом,NaN,NaN


In [18]:
# переименовываем имена колонок таблицы условий в совпадающие с именами в БД
Cond_df.rename(columns={'Доля учитываемого фонда': 'share',
                        'Возраст':'age',
                        'Аварийный':'is_alarm',
                        'Несущие стены':'wall_material', 
                        'Тип дома':'house_type',
                        'Этажность (от)':'floor_0', 
                        'Этажность (до)':'floor_1' }, 
               index=dict(zip(Cond_df.index,[i.replace('Условие ','cond') for i in Cond_df.index])), inplace = True)

In [19]:
print('Итоговая таблица условий выглядит следующим образом:')
Cond_df

Итоговая таблица условий выглядит следующим образом:


,share,age,is_alarm,wall_material,house_type,floor_0,floor_1
cond1,1.0,50.0,Нет,Деревянные,Многоквартирный дом,NaN,NaN
cond2,0.7,60.0,Нет,Панельные,Многоквартирный дом,1.0,5.0
cond3,0.5,80.0,Нет,Смешанные,Многоквартирный дом,NaN,NaN
cond4,0.5,60.0,Нет,Иные,Многоквартирный дом,NaN,NaN
cond5,0.3,100.0,Нет,Блочные,Многоквартирный дом,NaN,NaN
cond6,0.3,100.0,Нет,Кирпич,Многоквартирный дом,1.0,5.0
cond7,0.1,100.0,Нет,Кирпич,Многоквартирный дом,6.0,50.0
cond8,0.5,100.0,Нет,Панельные,Многоквартирный дом,6.0,50.0
cond9,0.1,150.0,Нет,Монолитные,Многоквартирный дом,NaN,NaN
cond10,NaN,NaN,Да,NaN,Многоквартирный дом,NaN,NaN


Полученные по панели для конкретного условия результаты выбытия корректируются на коэффициент:

(Жилая площадь, по состоянию на конец 2019 г.)/(суммарная жилая площадь в панели по МКД конкретного вида, где нет пропуска ни  по одному из параметров условий)

Жилая площадь, по состоянию на конец 2019 г. (млн кв. м.) (источник - https://fedstat.ru/indicator/57479):

In [20]:
HouseSqTot=2353.040248

## Расчитываем выбытия площадей ЖФ по определенным условиям

Создаем пустой датафрейм для вывода подсчетов выбытий ЖФ. Используя по очереди каждое условие - заполняем датафрейм. Информация о рассчитываемом в данный момент условии и успешном его завершении будет появляться ниже, под кодом.

In [21]:
w_cols=list(range(2015, 2036)) # задается период 2015 - 2035 гг.
idx=['cond'+str(i) for i in range(1,11)] 

In [22]:
fdf=pd.DataFrame(columns=w_cols, index=idx)

In [54]:
for i, row in Cond_df.iterrows():
    d_main=row.to_dict()
    
    print(i,'...')
    
    if pd.notna(d_main['age']):
        age=d_main['age']
        share=d_main['share']
        #print('age:',age,'share:',share)
        if pd.isna(d_main['floor_0']):
            d_body={k: d_main[k] for k in d_main if (pd.notna(d_main[k])) and k not in ['age', 'share']}
            query_age="(col-wdf['built_year']>={})".format(age)
            query_body = ' & '.join(['(wdf[{}]=={})'.format(repr(k),repr(v)) for k,v in d_body.items()])
            query=query_body+' & '+query_age
            #print(query)
        else:
            d_body={k: d_main[k] for k in d_main if pd.notna(d_main[k]) and k not in ['age','share','floor_0','floor_1']}
            d_floor={k: d_main[k] for k in ['floor_0','floor_1']}
            query_age="(col-wdf['built_year']>={})".format(age)
            query_body = ' & '.join(['(wdf[{}]=={})'.format(repr(k),repr(v)) for k,v in d_body.items()])
            query_floor = "(wdf['floor_count_max'].between({},{}))".format(d_floor['floor_0'],d_floor['floor_1'])
            query=query_body+' & '+query_floor+' & '+query_age
            d_body.update({'floor_count_max':'no matter'})
            #print(query)
    else:
        share=1
        d_body={k: d_main[k] for k in d_main if pd.notna(d_main[k])}
        query = ' & '.join(['(wdf[{}]=={})'.format(repr(k),repr(v)) for k,v in d_body.items()])
        #print(query)

    for col in fdf.columns:
        fdf.loc[i,col] = (wdf[eval(query)]['area_residential'].sum()/1000000) * HouseSqTot / (wdf.dropna(subset=d_body.keys())['area_residential'].sum()/1000000) * share
    print('условие посчитано')    
    
fdf.loc['Total']=fdf.sum()


cond1
age: 50.0 share: 1.0
(wdf['is_alarm']=='Нет') & (wdf['wall_material']=='Деревянные') & (wdf['house_type']=='Многоквартирный дом') & (col-wdf['built_year']>=50.0)
query done
cond2
age: 60.0 share: 0.7
(wdf['is_alarm']=='Нет') & (wdf['wall_material']=='Панельные') & (wdf['house_type']=='Многоквартирный дом') & (wdf['floor_count_max'].between(1.0,5.0)) & (col-wdf['built_year']>=60.0)
query done
cond3
age: 80.0 share: 0.5
(wdf['is_alarm']=='Нет') & (wdf['wall_material']=='Смешанные') & (wdf['house_type']=='Многоквартирный дом') & (col-wdf['built_year']>=80.0)
query done
cond4
age: 60.0 share: 0.5
(wdf['is_alarm']=='Нет') & (wdf['wall_material']=='Иные') & (wdf['house_type']=='Многоквартирный дом') & (col-wdf['built_year']>=60.0)
query done
cond5
age: 100.0 share: 0.3
(wdf['is_alarm']=='Нет') & (wdf['wall_material']=='Блочные') & (wdf['house_type']=='Многоквартирный дом') & (col-wdf['built_year']>=100.0)
query done
cond6
age: 100.0 share: 0.3
(wdf['is_alarm']=='Нет') & (wdf['wall_mate

In [55]:
print('Итоговая таблица выбытий (суммарные выбытия в строке Total):')
fdf

,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,...,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
cond1,9.34215,9.57903,9.82567,10.1007,10.3586,10.6438,10.8633,11.0806,11.2969,11.515,...,11.9524,12.1533,12.3855,12.62,12.9133,13.1651,13.4579,13.8009,14.1052,14.4564
cond2,0.465691,0.546456,0.713911,1.05665,1.60964,3.02294,5.74194,10.058,15.4078,21.7082,...,37.1337,45.4637,54.2845,62.6981,71.632,80.2909,89.5933,99.0261,108.428,117.472
cond3,0.589838,0.602893,0.646495,0.656133,0.671222,0.685506,0.692259,0.694228,0.695229,0.697849,...,0.742774,0.759767,0.802362,0.856007,0.953248,1.02691,1.17504,1.29271,1.4229,1.53944
cond4,1.18401,1.34731,1.52947,1.79454,2.09883,2.42196,2.76028,3.16659,3.59349,3.93737,...,4.52737,4.92572,5.56665,6.24012,6.80073,7.35003,8.05759,8.7048,9.2629,9.83749
cond5,0.0237507,0.0237507,0.0373752,0.0378104,0.0383036,0.0387219,0.0387571,0.0387571,0.0389858,0.0390482,...,0.0435001,0.0449365,0.0480546,0.0509389,0.0564542,0.0663619,0.0757525,0.0827552,0.091864,0.103348
cond6,3.38061,3.54668,4.75034,4.79139,4.79622,4.80563,4.80726,4.81521,4.82548,4.83963,...,4.95727,5.0763,5.25434,5.48265,5.85576,6.10499,6.41483,6.61429,6.86142,7.09411
cond7,0.599469,0.620356,0.675904,0.675904,0.675904,0.676409,0.676409,0.676409,0.677382,0.681832,...,0.701506,0.723425,0.752419,0.775276,0.787623,0.803222,0.819849,0.835917,0.853371,0.868685
cond8,0.0419674,0.0419674,0.0419674,0.0419674,0.0419674,0.0419674,0.0419674,0.0484751,0.0484751,0.0484751,...,0.0484751,0.0484751,0.0484751,0.049482,0.0511055,0.0511055,0.0528087,0.073758,0.073758,0.0749336
cond9,0.000723875,0.000723875,0.000723875,0.000723875,0.000723875,0.000771167,0.000771167,0.000771167,0.000771167,0.000771167,...,0.00101757,0.00101757,0.00101757,0.00101757,0.00136875,0.00136875,0.00136875,0.00136875,0.00136875,0.00136875
cond10,25.0925,25.0925,25.0925,25.0925,25.0925,25.0925,25.0925,25.0925,25.0925,25.0925,...,25.0925,25.0925,25.0925,25.0925,25.0925,25.0925,25.0925,25.0925,25.0925,25.0925


## Сверяем со старыми данными
этот раздел используется для проверки проведенных расчетов. Его не будет в финальной версии кода по данныой модели.

In [26]:
tdf=pd.read_excel('try.xlsx') #старая БД из Олиного экселя
tdf.replace(['Не заполнено','Нет данных','нет данных'], np.nan, inplace=True)
tdf.rename(columns={'FLR': 'floor_count_max'})

In [30]:
fdf2=pd.DataFrame(columns=w_cols, index=idx)

In [31]:
for i, row in Cond_df.iterrows():
    d_main=row.to_dict()
    
    print(i)
    
    if pd.notna(d_main['age']):
        age=d_main['age']
        share=d_main['share']
        print('age:',age,'share:',share)
        if pd.isna(d_main['floor_0']):
            d_body={k: d_main[k] for k in d_main if (pd.notna(d_main[k])) and k not in ['age', 'share']}
            query_age="(col-tdf['built_year']>={})".format(age)
            query_body = ' & '.join(['(tdf[{}]=={})'.format(repr(k),repr(v)) for k,v in d_body.items()])
            query=query_body+' & '+query_age
            print(query)
        else:
            d_body={k: d_main[k] for k in d_main if pd.notna(d_main[k]) and k not in ['age','share','floor_0','floor_1']}
            d_floor={k: d_main[k] for k in ['floor_0','floor_1']}
            query_age="(col-tdf['built_year']>={})".format(age)
            query_body = ' & '.join(['(tdf[{}]=={})'.format(repr(k),repr(v)) for k,v in d_body.items()])
            query_floor = "(tdf['floor_count_max'].between({},{}))".format(d_floor['floor_0'],d_floor['floor_1'])
            query=query_body+' & '+query_floor+' & '+query_age
            d_body.update({'floor_count_max':'no matter'})
            print(query)
    else:
        share=1
        d_body={k: d_main[k] for k in d_main if pd.notna(d_main[k])}
        query = ' & '.join(['(tdf[{}]=={})'.format(repr(k),repr(v)) for k,v in d_body.items()])
        print(query)

    for col in fdf2.columns:
        fdf2.loc[i,col] = (tdf[eval(query)]['area_residential'].sum()/1000000) * HouseSqTot / (tdf.dropna(subset=d_body.keys())['area_residential'].sum()/1000000) * share
    print('Условие посчитано')    
    
fdf2.loc['Total']=fdf2.sum()

{'age': 50.0, 'is_alarm': 'Нет', 'wall_material': 'Деревянные', 'house_type': 'Многоквартирный дом'}
{'age': 60.0, 'is_alarm': 'Нет', 'wall_material': 'Панельные', 'house_type': 'Многоквартирный дом', 'floor_0': 1.0, 'floor_1': 5.0}
{'age': 80.0, 'is_alarm': 'Нет', 'wall_material': 'Смешанные', 'house_type': 'Многоквартирный дом'}
{'age': 60.0, 'is_alarm': 'Нет', 'wall_material': 'Иные', 'house_type': 'Многоквартирный дом'}
{'age': 100.0, 'is_alarm': 'Нет', 'wall_material': 'Блочные', 'house_type': 'Многоквартирный дом'}
{'age': 100.0, 'is_alarm': 'Нет', 'wall_material': 'Кирпич', 'house_type': 'Многоквартирный дом', 'floor_0': 1.0, 'floor_1': 5.0}
{'age': 100.0, 'is_alarm': 'Нет', 'wall_material': 'Кирпич', 'house_type': 'Многоквартирный дом', 'floor_0': 6.0, 'floor_1': 50.0}
{'age': 100.0, 'is_alarm': 'Нет', 'wall_material': 'Панельные', 'house_type': 'Многоквартирный дом', 'floor_0': 6.0, 'floor_1': 50.0}
{'age': 150.0, 'is_alarm': 'Нет', 'wall_material': 'Монолитные', 'house_type':

In [32]:
fdf2

,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,...,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
cond1,9.75584,10.0159,10.2745,10.5567,10.8274,11.133,11.3713,11.6041,11.8306,12.0595,...,12.5226,12.74,12.9892,13.239,13.554,13.8217,14.1368,14.5031,14.8419,15.2285
cond2,0.429688,0.514275,0.682917,1.03044,1.58981,2.99843,5.72323,10.091,15.5893,22.0877,...,37.9546,46.4623,55.2932,64.0229,73.327,82.2732,91.8742,101.481,111.047,120.351
cond3,0.401962,0.41562,0.46122,0.470806,0.484848,0.499808,0.506434,0.508034,0.509083,0.511827,...,0.55825,0.573978,0.61865,0.673562,0.77524,0.853166,1.00809,1.12951,1.26659,1.3848
cond4,1.25743,1.4324,1.64027,1.93659,2.26343,2.63197,3.05041,3.51422,3.98857,4.37425,...,5.02392,5.46282,6.1536,6.89359,7.49763,8.10093,8.87183,9.59903,10.268,10.9242
cond5,0.00924062,0.00924062,0.0209015,0.0213576,0.0218743,0.0223125,0.0223494,0.0223494,0.0223494,0.0224146,...,0.0233904,0.0243784,0.0264416,0.0275954,0.0333721,0.043177,0.0530158,0.060165,0.0695504,0.0814605
cond6,0.463528,0.496585,1.22558,1.26348,1.26537,1.27516,1.27637,1.28161,1.2901,1.29835,...,1.3524,1.41689,1.52826,1.68507,1.93736,2.14469,2.45408,2.65802,2.90243,3.13439
cond7,0.00265659,0.00362126,0.00543468,0.00543468,0.00543468,0.00543468,0.00543468,0.00543468,0.00543468,0.00543468,...,0.00543468,0.00575255,0.00575255,0.010491,0.0119581,0.0273982,0.04482,0.0616487,0.07676,0.091779
cond8,0.0185683,0.0185683,0.0185683,0.0185683,0.0185683,0.0185683,0.0185683,0.0253839,0.0253839,0.0253839,...,0.0253839,0.0253839,0.0253839,0.0253839,0.0253839,0.0253839,0.0271677,0.0327473,0.0327473,0.0339788
cond9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cond10,24.8407,24.8407,24.8407,24.8407,24.8407,24.8407,24.8407,24.8407,24.8407,24.8407,...,24.8407,24.8407,24.8407,24.8407,24.8407,24.8407,24.8407,24.8407,24.8407,24.8407


График сравнения 

In [56]:
s   = fdf.loc['Total']
s2  = fdf2.loc['Total']
asa = s.plot.line(figsize=(14, 5), linewidth=4, alpha=0.5, title='Общее выбытие МКД', color='g')
bsa = s2.plot.line(ax=asa, linewidth=4, alpha=0.5, color='r', grid=True)
plt.tight_layout()
plt.show()

NameError: name 'fdf2' is not defined